In [1]:
import pandas as pd
import numpy as np

from horta_exp.introns.fetch_data import get_intron_events

from limix_genetics import hitsplot
from bokeh.io import push_notebook, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure, show, output_file
from bokeh.charts import BoxPlot, Bar, Histogram
from bokeh.layouts import gridplot

FILE = {'real': 'chrom_real_all.pkl', 'null': 'chrom_null_all.pkl'}
output_notebook()

Loading BokehJS ...

In [2]:
def convert_and_save(filename):
    introns = get_intron_events()
    df = pd.read_pickle(filename).set_index(['gene', 'intron']).sort_index()
    introns = introns.loc[df.index.drop_duplicates()]

    introns.to_pickle('introns_%s.pkl' % filename)
    df.to_pickle('df_%s.pkl' % filename)
    
    return (introns, df)

In [3]:
introns = dict()
df = dict()

if True:
    for n in ['real', 'null']:
        introns[n] = pd.read_pickle('introns_%s.pkl' % FILE[n])
        df[n] = pd.read_pickle('df_%s.pkl' % FILE[n])
else:
    for n in ['real', 'null']:
        (introns[n], df[n]) = convert_and_save(FILE[n])

In [ ]:
npairs = len(df['real'].index.unique())
print("Number of gene-intron pairs: %d" % npairs)

# Bonferroni correction within gene-introns and aggregate the p-values

In [ ]:
def bonferroni_and_aggregate(df):
    def pval_agg(x):
        return np.clip(np.min(x) * len(x), 0, 1)

    return df.groupby(level=[0, 1]).agg({
        'lmm-pval': pval_agg,
        'lmm-rank-pval': pval_agg,
        'qep-pval': pval_agg
    })

for n in ['real', 'null']:
    df[n] = bonferroni_and_aggregate(df[n])

In [ ]:
from IPython.display import display
for n in ['real', 'null']:
    display(df[n].head())

In [ ]:
def method_column(df):
    newdata = pd.DataFrame(columns=('pval', 'method'))

    def append_df(data, df, name):
        pval = df['%s-pval' % name]
        return pd.concat([data, pd.DataFrame({'pval': pval, 'method': name})])
        return data

    newdata = append_df(newdata, df, 'lmm')
    newdata = append_df(newdata, df, 'lmm-rank')
    newdata = append_df(newdata, df, 'qep')
    return newdata

def set_index(data):
    data = method_column(data)
    data.reset_index(inplace=True)
    data['gene'] = data['index'].apply(lambda x: x[0])
    data['intron'] = data['index'].apply(lambda x: int(x[1]))
    del data['index']
    data.set_index(['gene', 'intron'], inplace=True)
    data.sort_index(inplace=True)
    return data

DF = dict()
for n in ['real', 'null']:
    DF[n] = set_index(df[n])
    DF[n] = DF[n].rename(columns={'method':'label', 'pval':'p-value'})

In [ ]:
for n in ['real', 'null']:
    display(DF[n].head())

In [ ]:
group = introns['real'].groupby(level=[0, 1]).agg({
    'ntri': {
        'max': 'max',
        'min': 'min',
        'median': 'median',
        'var': 'var',
        'mean': 'mean',
    }
})
group.sort_index(inplace=True)

In [ ]:
group.head()

In [ ]:
def plot_grid(DF, ntri, introns, STAT):
    
    size = 400
    colors={'lmm': 'blue', 'qep': 'red', 'lmm-rank': 'green'}
    
    df = DF['real'].loc[ntri.index].copy()
    df['p-value'] = np.clip(df['p-value'] * npairs, 0, 1)
    
    f0 = hitsplot(df,
                  colors=colors,
         perc=True, width=size, height=size, min_threshold=1e-5, max_threshold=1e-2,
        show=False)
    
    df = DF['null'].loc[ntri.index].copy()
    df['p-value'] = np.clip(df['p-value'] * npairs, 0, 1)
    
    f1 = hitsplot(df, colors=colors,
         perc=True, width=size, height=size, min_threshold=1e-5, max_threshold=1e-2,
        show=False)

    
    def choice(vals):
        return np.random.RandomState(0).choice(vals, min(5000, len(vals)))
    
    ntris = choice(introns.loc[ntri.index]['ntri'].values)
    h0 = Histogram(ntris, title="Number of trials - across samples",
                   tools=['save'], xlabel='number of trials',
                   width=size, height=size)
    
    values = choice(ntri[STAT].values)
    h1 = Histogram(values, title="%s number of trials - across gene-intron pairs" % STAT,
                   tools=['save'], xlabel='number of trials', width=size, height=size)
    
    
    show(gridplot([[f0, f1], [h0, h1]]))

In [ ]:
ntri = group['ntri'].copy()
ntri = ntri[ntri['min'] > 30]
ntri = ntri[ntri['var'] < ntri['var'].quantile(0.10)]
plot_grid(DF, ntri, introns['real'], 'min')

In [ ]:
ntri = group['ntri'].copy()
ntri = ntri[ntri['min'] > 30]
ntri = ntri[ntri['var'] < ntri['var'].quantile(0.20)]
plot_grid(DF, ntri, introns['real'], 'min')

In [ ]:
ntri = group['ntri'].copy()
ntri = ntri[ntri['min'] > 30]
ntri = ntri[ntri['var'] < ntri['var'].quantile(0.30)]
plot_grid(DF, ntri, introns['real'], 'min')

In [ ]:
ntri = group['ntri'].copy()
ntri = ntri[ntri['min'] > 30]
ntri = ntri[ntri['var'] < ntri['var'].quantile(0.40)]
plot_grid(DF, ntri, introns['real'], 'min')